In [ ]:
import csv
import time
import os

import pandas as pd
from PIL import ImageFile
from PIL import Image
import torch
start_time = time.time()
ImageFile.LOAD_TRUNCATED_IMAGES = True
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
folder="Car2/"
#Car for correctly classified car images
#Truck for correctly classified truck images
#Car2 for incorrectly classified car images
#Truck2 for incorrectly classified truck images
path_data = folder # Here is your path
count=0
for dir_name, _, file_list in os.walk(path_data):
    for file in sorted(file_list):
        if file :
            count=count+1
            image=Image.open(os.path.join(dir_name, file))
            inputs = processor(text=["a photo of a truck", "a photo of a car"], images=image, return_tensors="pt", padding=True)
            #outputs = model(**inputs)
            #print(outputs.logits_per_image)
            #logits_per_image = outputs.logits_per_image # this is the image-text similarity score
            #probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
            #print(os.path.join(dir_name, file),probs[0].argmax()) # Prints File Names with Predicted Label
            #######################################################################################################################
            model.vision_model.encoder.layers[18].layer_norm2.register_forward_hook(get_features("feats"))
            device = torch.device("cpu")
            PREDS = []
            FEATS = []
            # placeholder for batch features
            features = {}
            # loop through batches # move to device
            inputs = inputs.to(device)
            # forward pass [with feature extraction]
            preds = model(**inputs)
            label=preds.logits_per_image.softmax(dim=1)[0].argmax()
            print(label)
            #print(os.path.join(dir_name, file),label)
            ###########################WRITES FEATURES TO CSV######################################################
            #for name, parameter in model.named_parameters(): # Uncomment for printing all Model Parameter Names
            #   print(name)
            #print(preds.logits_per_image) # Print final label output
            #print(features["feats"].flatten()) # Print feature Values
            with open("label.csv", "a",newline='') as fp:
                # (0: cars incorrectly classified)
                # (1: cars correctly classified)
                # (2: trucks correctly classified)
                # (3: trucks incorrectly classified)
               wr = csv.writer(fp, dialect='excel')
               if folder == "Car2/" and label == 0:
                   wr.writerow('0')
               if folder == "Car/" and label == 1:
                   wr.writerow('1')
               if folder=="Truck/" and label==0:
                   wr.writerow('2')
               if folder == "Truck2/" and label == 1:
                    wr.writerow('3')

            with open("features.csv", "a") as fp:
                wr = csv.writer(fp, dialect='excel')
                wr.writerow(features["feats"].flatten().tolist())
            df = pd.read_csv('features.csv')
            print(df.shape)
            ########################################################################################################



tensor(0)
(0, 263168)
tensor(0)
(1, 263168)
tensor(0)
(2, 263168)
tensor(0)
(3, 263168)
tensor(0)
(4, 263168)
tensor(0)
(5, 263168)
tensor(0)
(6, 263168)
tensor(0)
(7, 263168)
tensor(0)
(8, 263168)
tensor(0)
(9, 263168)
tensor(0)
(10, 263168)
tensor(0)
(11, 263168)
tensor(0)
(12, 263168)
tensor(0)
(13, 263168)
tensor(0)
(14, 263168)
tensor(0)
(15, 263168)
tensor(0)
(16, 263168)
tensor(0)
(17, 263168)
tensor(0)
(18, 263168)
tensor(0)
(19, 263168)
tensor(0)
(20, 263168)
tensor(0)
(21, 263168)
tensor(0)
(22, 263168)
tensor(0)
(23, 263168)
tensor(0)
(24, 263168)
tensor(0)
(25, 263168)
tensor(0)
(26, 263168)
tensor(0)
(27, 263168)
tensor(0)
(28, 263168)
tensor(0)
(29, 263168)
tensor(0)
(30, 263168)
tensor(0)
(31, 263168)
tensor(0)
(32, 263168)
tensor(0)
(33, 263168)
tensor(0)
(34, 263168)
tensor(0)
(35, 263168)
tensor(0)
(36, 263168)
tensor(0)
(37, 263168)
tensor(0)
(38, 263168)
tensor(0)
(39, 263168)
tensor(0)
(40, 263168)


In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))